In [1]:
#import needed package
#---------------------------------------------------------
import pandas as pd
import numpy as np
import sklearn
import autosklearn
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import datetime
import autosklearn.classification
from sklearn.preprocessing import LabelEncoder #OrdinalEncoder

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#print versions
#---------------------------------------------------------
print('The scikit-learn version is {}.'.format(sklearn.__version__))
print('The Auto scikit-learn version is {}.'.format(autosklearn.__version__))

The scikit-learn version is 0.19.2.
The Auto scikit-learn version is 0.5.2.


In [3]:
#Load Data from CSV
#--------------------------------------------------------
filepath = '/data/home/eissa/Downloads/datasets_91-BayesianNetworkGenerator_mfeat-fourier_small.csv'
rawdata = pd.read_csv( filepath ,  sep=','  ,  low_memory=False )
rawdata.shape

(1000000, 77)

In [4]:
#Categorical feature encoding
#--------------------------------------------------------
label = 'label'
col = [label]
for i in range(76):
    col.append('c' + str(i))
rawdata.columns = col
rawdata.head(10)
le = LabelEncoder()
for column in rawdata: 
    rawdata[column] = le.fit_transform(rawdata[column])

In [ ]:
rawdata.head(10)

In [ ]:
#Label casting
#--------------------------------------------------------
rawdata[[label]] = rawdata[[label]].astype(int)

In [ ]:
# split data into Testing and training
# ---------------------------------------------------------------------------------------------------------------------
np.random.seed(1234)
msk = np.random.rand(len(rawdata)) < 0.8
training_data = rawdata[msk]
Testing_data = rawdata[~msk]

X_train = training_data.loc[:, training_data.columns != label]
Y_train = training_data.loc[:, training_data.columns == label]

X_test = Testing_data.loc[:, Testing_data.columns != label]
Y_test = Testing_data.loc[:, Testing_data.columns == label]

In [ ]:
# auto ML
# ---------------------------------------------------------------------------------------------------------------------
t1 = datetime.datetime.now()
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=300, 
                                                          per_run_time_limit=100,
                                                          ensemble_memory_limit=10000, 
                                                          seed=1, ml_memory_limit=10000)
automl.fit(X_train, Y_train)
y_hat = automl.predict(X_test)
t2 = datetime.datetime.now()
print("Time:" + str((t2 - t1).total_seconds()))
print("Accuracy score", accuracy_score(Y_test, y_hat))


In [ ]:
automl.cv_results_

In [ ]:
automl.show_models(

In [ ]:
automl.sprint_statistics()